In [5]:
import pandas as pd
import geopandas as gpd

# CSV 파일 읽기
df = pd.read_csv('2022년 외국인인구.csv', encoding='utf-8')

# 1) 시군구 코드와 이름 컬럼 만들기
df['CD'] = df['CD'].astype(str)
df['SGG_CD'] = df['CD'].str[:5]
df['SGG_NM'] = df[df['CD'].str.len() == 5]['NM']
df['SGG_NM'] = df.groupby('SGG_CD')['SGG_NM'].transform('first')

# 2) 컬럼 이름 변경
df = df.rename(columns={'CD': 'ADM_CD', 'NM': 'ADM_NM'})

# 3) 피봇팅
pivot_columns = {
    '합계': 'TOTAL',
    '한국국적을 가지지 않은 자': 'FOREIGN',
    '한국국적을 취득한 자': 'NATURALIZED',
    '외국인주민자녀(출생)': 'FOR_CHILD'
}

# VALUE 열의 '*' 값을 0으로 변경
df['VALUE'] = df['VALUE'].replace('*', '0')
df_pivot = df.pivot(index=['ADM_CD', 'ADM_NM', 'SGG_CD', 'SGG_NM'], 
                    columns='CMN', 
                    values='VALUE')[pivot_columns.keys()]
df_pivot.columns = pivot_columns.values()
df_pivot = df_pivot.reset_index()

# 4) ADM_CD가 7자리인 값만 남기기(읍면동 데이터만 남기기)
df_result = df_pivot[df_pivot['ADM_CD'].str.len() == 7].copy()

# 5) 추가 작업
# 5-1) ADM_NM이 보라동이면 SGG_NM에 '용인시 기흥구'를 추가
df_result.loc[df_result['ADM_NM'] == '보라동', 'SGG_NM'] = '용인시 기흥구'

# 5-2) SGG_NM에 세종시를 '세종특별자치시'로 변경
df_result['SGG_NM'] = df_result['SGG_NM'].replace('세종시', '세종특별자치시')

# 5-3) ADM_NM 일원2동을 '개포3동'으로 변경
df_result['ADM_NM'] = df_result['ADM_NM'].replace('일원2동', '개포3동')

# 5-4) ADM_NM 가야제1동을 '가야1동'으로 변경
df_result['ADM_NM'] = df_result['ADM_NM'].replace('가야제1동', '가야1동')

# 5-4) ADM_NM 당감제1동을 '당감1동'으로 변경
df_result['ADM_NM'] = df_result['ADM_NM'].replace('당감제1동', '당감1동')

# 결과 확인
print(df_result.head())

# CSV 파일로 저장
df_result.to_csv('processed_foreign_population_2022.csv', index=False, encoding='utf-8-sig')

    ADM_CD ADM_NM SGG_CD SGG_NM TOTAL FOREIGN NATURALIZED FOR_CHILD
1  1101053    사직동  11010    종로구   378     323          22        33
2  1101054    삼청동  11010    종로구    83      70           6         7
3  1101055    부암동  11010    종로구   445     403          17        25
4  1101056    평창동  11010    종로구   409     313          21        75
5  1101057    무악동  11010    종로구    86      51          16        19


In [11]:
# 이전에 저장한 CSV 파일 읽기
df_csv = pd.read_csv('processed_foreign_population_2022.csv')

# SHP 파일 읽기
gdf_shp = gpd.read_file('20224Q읍면동경계_2022_총조사총인구.shp')

# KEY 관련 컬럼을 문자열로 변환
for df in [df_csv, gdf_shp]:
    for col in ['SGG_CD', 'SGG_NM', 'ADM_NM']:
        if col in df.columns:
            df[col] = df[col].astype(str)

# 키 값 생성
df_csv['KEY'] = df_csv['SGG_NM'] + '_' + df_csv['ADM_NM']
gdf_shp['KEY'] = gdf_shp['SGG_NM'] + '_' + gdf_shp['ADM_NM']

# 키 값의 유니크 여부 확인 및 중복 키에 대해서만 SGG_CD 추가
if len(df_csv['KEY'].unique()) == len(df_csv):
    print("KEY is unique in CSV file")
else:
    print("Some KEYs are not unique in CSV file, adding SGG_CD to duplicates")
    # 중복된 키 찾기
    duplicated_keys = df_csv[df_csv['KEY'].duplicated(keep=False)]
    
    # 중복된 키에 대해서만 SGG_CD 추가
    df_csv.loc[duplicated_keys.index, 'KEY'] = df_csv.loc[duplicated_keys.index, 'SGG_CD'] + '_' + df_csv.loc[duplicated_keys.index, 'KEY']
    
    # SHP 파일에도 동일한 처리
    gdf_shp['KEY'] = gdf_shp.apply(lambda row: row['SGG_CD'] + '_' + row['KEY'] if row['KEY'] in duplicated_keys['KEY'].values else row['KEY'], axis=1)

# 조인 수행
gdf_result = gdf_shp.merge(df_csv, on='KEY', how='outer', indicator=True, suffixes=('_shp', '_csv'))

# 결과 데이터프레임의 컬럼 확인
print("Columns in the result dataframe:")
print(gdf_result.columns)

# SHP 기준으로 매칭되지 않은 결과 찾기
unmatched_shp = gdf_result[gdf_result['_merge'] == 'left_only']

# CSV 기준으로 매칭되지 않은 결과 찾기
unmatched_csv = gdf_result[gdf_result['_merge'] == 'right_only']

# 매칭되지 않은 결과 텍스트 파일로 저장
with open('unmatched_results.txt', 'w', encoding='utf-8') as f:
    f.write("Unmatched from SHP file:\n")
    unmatched_shp[['KEY', 'SGG_NM_shp', 'ADM_NM_shp']].to_csv(f, sep='\t', index=False)
    f.write("\nUnmatched from CSV file:\n")
    unmatched_csv[['KEY', 'SGG_NM_csv', 'ADM_NM_csv']].to_csv(f, sep='\t', index=False)


# 불필요한 컬럼 제거 및 컬럼명 정리
columns_to_keep = ['geometry', 'SGG_CD_shp', 'SGG_NM_shp', 'ADM_NM_shp', 'ADM_CD_shp', 'POPULATION', 'TOTAL', 'FOREIGN', 'NATURALIZED', 'FOR_CHILD', 'ADM_CD_csv']
columns_to_keep_CSV = ['SGG_CD_shp', 'SGG_NM_shp', 'ADM_NM_shp', 'ADM_CD_shp', 'POPULATION', 'TOTAL', 'FOREIGN', 'NATURALIZED', 'FOR_CHILD', 'ADM_CD_csv']

gdf_result = gdf_result[columns_to_keep]
gdf_result_csv = gdf_result[columns_to_keep_CSV]
gdf_result.columns = ['geometry', 'SGG_CD', 'SGG_NM', 'ADM_NM', 'ADM_CD', 'POPULATION', 'TOT_FOREIGN', 'FOREIGN', 'NATURALIZED', 'FOR_CHILD', 'REF_ADM_CD']
gdf_result_csv.columns = ['SGG_CD', 'SGG_NM', 'ADM_NM', 'ADM_CD', 'POPULATION', 'TOT_FOREIGN', 'FOREIGN', 'NATURALIZED', 'FOR_CHILD', 'REF_ADM_CD']


# 결과를 CSV와 SHP 파일로 저장
gdf_result_csv.to_csv('20224Q읍면동경계_2022_총조사총인구_외국인인구.csv', index=False, encoding='utf-8-sig')
gdf_result.to_file('20224Q읍면동경계_2022_총조사총인구_외국인인구.shp', encoding='euc-kr')

print("처리가 완료되었습니다. 결과는 '20224Q읍면동경계_2022_총조사총인구_외국인인구.csv'와 '20224Q읍면동경계_2022_총조사총인구_외국인인구.shp'에 저장되었습니다.")
print(f"SHP 파일 기준 매칭되지 않은 {len(unmatched_shp)} 개와 CSV 파일 기준 매칭되지 않은 {len(unmatched_csv)} 개의 결과가 'unmatched_results.txt'에 저장되었습니다.")

Some KEYs are not unique in CSV file, adding SGG_CD to duplicates
Columns in the result dataframe:
Index(['SGG_CD_shp', 'SGG_NM_shp', 'ADM_NM_shp', 'ADM_CD_shp', 'POPULATION',
       'geometry', 'KEY', 'ADM_CD_csv', 'ADM_NM_csv', 'SGG_CD_csv',
       'SGG_NM_csv', 'TOTAL', 'FOREIGN', 'NATURALIZED', 'FOR_CHILD', '_merge'],
      dtype='object')


C:\Users\20180094\AppData\Local\Temp\ipykernel_9664\2540633799.py:64: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf_result.to_file('20224Q읍면동경계_2022_총조사총인구_외국인인구.shp', encoding='euc-kr')


처리가 완료되었습니다. 결과는 '20224Q읍면동경계_2022_총조사총인구_외국인인구.csv'와 '20224Q읍면동경계_2022_총조사총인구_외국인인구.shp'에 저장되었습니다.
SHP 파일 기준 매칭되지 않은 5 개와 CSV 파일 기준 매칭되지 않은 0 개의 결과가 'unmatched_results.txt'에 저장되었습니다.
